## Test generator & checker for class Matrix

In [4]:
import numpy as np
from matrix import Matrix
import random
import sys

In [13]:
def assert_close(a, b, tolerance=1e-10):
    if isinstance(a, (int, float)) and isinstance(b, (int, float)):
        assert abs(a - b) < tolerance, f"{a} is not close to {b}"
    elif isinstance(a, list) and isinstance(b, list):
        assert len(a) == len(b), "Lists have different lengths"
        for i in range(len(a)):
            assert_close(a[i], b[i], tolerance)
    elif isinstance(a, Matrix) and isinstance(b, np.ndarray):
        assert a.rows == b.shape[0] and a.cols == b.shape[1], "Matrices have different shapes"
        for i in range(a.rows):
            for j in range(a.cols):
                assert_close(a.matrix[i][j], b[i, j], tolerance)
    else:
        raise ValueError("Unsupported types for comparison")

def random_matrix(rows, cols, min_val=-10, max_val=10):
    return Matrix([[random.uniform(min_val, max_val) for _ in range(cols)] for _ in range(rows)])

def print_matrix(matrix):
    for row in matrix:
        print(" ".join(f"{x:8.4f}" for x in row))

def test_matrix_operations(num_tests=50):
    print(f"\nTesting matrix operations ({num_tests} tests):")
    for i in range(num_tests):
        rows, cols = random.randint(2, 5), random.randint(2, 5)
        A = random_matrix(rows, cols)
        B = random_matrix(rows, cols)
        scalar = random.uniform(-10, 10)

        print(f"\nTest {i+1}:")
        print("Matrix A:")
        print_matrix(A.matrix)
        print("Matrix B:")
        print_matrix(B.matrix)
        print(f"Scalar: {scalar:.4f}")

        # Test addition
        print("Testing addition (A + B)...")
        C = A + B
        assert_close(C, np.array(A.matrix) + np.array(B.matrix))
        print("Addition passed.")

        # Test subtraction
        print("Testing subtraction (A - B)...")
        D = A - B
        assert_close(D, np.array(A.matrix) - np.array(B.matrix))
        print("Subtraction passed.")

        # Test multiplication (matrix-scalar)
        print(f"Testing scalar multiplication (A * {scalar:.4f})...")
        E = A * scalar
        assert_close(E, np.array(A.matrix) * scalar)
        print("Scalar multiplication passed.")

        # Test division (matrix-scalar)
        print(f"Testing scalar division (A / {scalar:.4f})...")
        F = A / scalar
        assert_close(F, np.array(A.matrix) / scalar)
        print("Scalar division passed.")

    print(f"\nAll {num_tests} matrix operations tests passed.")

def test_matrix_properties(num_tests=50):
    print(f"\nTesting matrix properties ({num_tests} tests):")
    for i in range(num_tests):
        size = random.randint(2, 5)
        A = random_matrix(size, size)
        np_A = np.array(A.matrix)

        print(f"\nTest {i+1}:")
        print("Matrix A:")
        print_matrix(A.matrix)

        # Test transpose
        print("Testing transpose...")
        assert_close(A.transpose(), np_A.T)
        print("Transpose passed.")

        # Test determinant
        print("Testing determinant...")
        our_det = A.determinant()
        np_det = np.linalg.det(np_A)
        print(f"Our determinant: {our_det:.4f}")
        print(f"NumPy determinant: {np_det:.4f}")
        assert_close(our_det, np_det)
        print("Determinant passed.")

        # Test inverse (only for invertible matrices)
        if abs(np.linalg.det(np_A)) > 1e-10:
            print("Testing inverse...")
            A_inv = A.inverse()
            np_A_inv = np.linalg.inv(np_A)
            assert_close(A_inv, np_A_inv)
            print("Inverse passed.")
        else:
            print("Matrix is not invertible, skipping inverse test.")

        # Test rank
        print("Testing rank...")
        our_rank = A.rank()
        np_rank = np.linalg.matrix_rank(np_A)
        print(f"Our rank: {our_rank}")
        print(f"NumPy rank: {np_rank}")
        assert our_rank == np_rank
        print("Rank passed.")

        # Test symmetry
        print("Testing symmetry...")
        our_symmetry = A.is_symmetric()
        np_symmetry = np.allclose(np_A, np_A.T)
        print(f"Our symmetry check: {our_symmetry}")
        print(f"NumPy symmetry check: {np_symmetry}")
        assert our_symmetry == np_symmetry
        print("Symmetry check passed.")

    print(f"\nAll {num_tests} matrix properties tests passed.")

def test_eigenvalues_and_vectors(num_tests=50):
    print(f"\nTesting eigenvalues and eigenvectors ({num_tests} tests):")
    for i in range(num_tests):
        size = random.randint(2, 5)
        A = random_matrix(size, size)
        np_A = np.array(A.matrix)

        print(f"\nTest {i+1}:")
        print("Matrix A:")
        print_matrix(A.matrix)

        # Test eigenvalues
        print("Testing eigenvalues...")
        our_eigenvalues = sorted(A.eigenvalues(), key=lambda x: x.evalf())
        np_eigenvalues = sorted(np.linalg.eigvals(np_A))
        print("Our eigenvalues:", [f"{e.evalf():.4f}" for e in our_eigenvalues])
        print("NumPy eigenvalues:", [f"{e:.4f}" for e in np_eigenvalues])
        assert_close(our_eigenvalues, np_eigenvalues)
        print("Eigenvalues passed.")

        # Test eigenvectors
        print("Testing eigenvectors...")
        our_eigenvectors = A.eigenvectors()
        np_eigenvalues, np_eigenvectors = np.linalg.eig(np_A)

        # Compare the span of eigenvectors
        our_span = Matrix(our_eigenvectors).rank()
        np_span = np.linalg.matrix_rank(np_eigenvectors)
        print(f"Our eigenvector span: {our_span}")
        print(f"NumPy eigenvector span: {np_span}")
        assert our_span == np_span
        print("Eigenvectors passed.")

    print(f"\nAll {num_tests} eigenvalues and eigenvectors tests passed.")

def test_matrix_decompositions(num_tests=50):
    print(f"\nTesting matrix decompositions ({num_tests} tests):")
    for i in range(num_tests):
        size = random.randint(2, 5)
        A = random_matrix(size, size)
        np_A = np.array(A.matrix)

        print(f"\nTest {i+1}:")
        print("Matrix A:")
        print_matrix(A.matrix)

        # Test diagonalization (only for diagonalizable matrices)
        print("Testing diagonalization...")
        try:
            P, D = A.diagonalize()
            
            # Verify A = PDP^-1
            result = P * D * P.inverse()
            assert_close(result, np_A)
            print("Diagonalization passed.")
        except ValueError as e:
            print(f"Matrix is not diagonalizable: {e}")

    print(f"\nAll {num_tests} matrix decompositions tests completed.")

def test_matrix_norms(num_tests=50):
    print(f"\nTesting matrix norms ({num_tests} tests):")
    for i in range(num_tests):
        rows, cols = random.randint(2, 5), random.randint(2, 5)
        A = random_matrix(rows, cols)
        np_A = np.array(A.matrix)

        print(f"\nTest {i+1}:")
        print("Matrix A:")
        print_matrix(A.matrix)

        # Test Frobenius norm
        print("Testing Frobenius norm...")
        our_norm = A.norm()
        np_norm = np.linalg.norm(np_A, 'fro')
        print(f"Our Frobenius norm: {our_norm:.4f}")
        print(f"NumPy Frobenius norm: {np_norm:.4f}")
        assert_close(our_norm, np_norm)
        print("Frobenius norm passed.")

        # Test vector norm
        print("Testing vector norm...")
        v = [random.uniform(-10, 10) for _ in range(random.randint(2, 5))]
        our_vector_norm = Matrix.vector_norm(v)
        np_vector_norm = np.linalg.norm(np.array(v))
        print(f"Vector: {v}")
        print(f"Our vector norm: {our_vector_norm:.4f}")
        print(f"NumPy vector norm: {np_vector_norm:.4f}")
        assert_close(our_vector_norm, np_vector_norm)
        print("Vector norm passed.")

    print(f"\nAll {num_tests} matrix norms tests passed.")

def run_all_tests():
    random.seed(42)  # For reproducibility
    np.random.seed(42)

    test_matrix_operations()
    test_matrix_properties()
    test_eigenvalues_and_vectors()
    test_matrix_decompositions()
    test_matrix_norms()
    print("\nAll random tests passed successfully!")

In [14]:
run_all_tests()


Testing matrix operations (50 tests):

Test 1:
Matrix A:
  4.8310  -5.1022
 -7.2092  -7.9501
Matrix B:
  4.8134   0.9073
  1.8099  -9.3643
Scalar: -8.1261
Testing addition (A + B)...
Addition passed.
Testing subtraction (A - B)...
Subtraction passed.
Testing scalar multiplication (A * -8.1261)...
Scalar multiplication passed.
Testing scalar division (A / -8.1261)...
Scalar division passed.

Test 2:
Matrix A:
  1.2249   4.3204
  4.0265  -1.6096
 -1.0158  -4.4362
Matrix B:
  7.3860   5.1761
 -6.8068  -1.5477
 -4.4426  -5.6937
Scalar: 5.2699
Testing addition (A + B)...
Addition passed.
Testing subtraction (A - B)...
Subtraction passed.
Testing scalar multiplication (A * 5.2699)...
Scalar multiplication passed.
Testing scalar division (A / 5.2699)...
Scalar division passed.

Test 3:
Matrix A:
 -2.4015  -2.8204
 -3.1209  -4.7096
Matrix B:
 -9.1310  -0.8115
 -7.5035   8.4459
Scalar: -8.4240
Testing addition (A + B)...
Addition passed.
Testing subtraction (A - B)...
Subtraction passed.
Testi

ValueError: Matrix is not invertible